In [60]:
import random
import string
import numpy as np
import random as rd

def get_random_string(length=28):
    # choose from all lowercase letter
    letters = string.ascii_letters
    return ''.join(random.choice(letters) for i in range(length))

# se ficar ruim, uma possível melhora é 
def add_random_asterisk(word):
    number_of_asterisks = random.randint(0,len(word))
    for asterisk in range(number_of_asterisks):
        random_idx = random.randint(0,len(word)-1)
        word_to_list = list(word)
        word_to_list[random_idx] = '*'
        word = "".join(word_to_list)
    return word
                          
def generate_pop(pop_size):
    pop = []
    for being in range(pop_size):
        pop.append(add_random_asterisk(get_random_string()))
    return pop

def word_to_integer_vector(word):
    return list(map(lambda x: ord(x),list(word)))

def evaluate_loss_function(word, target="METHINKS*IT*IS*LIKE*A*WEASEL"):
    print(f"Word:   {word}\n")
    print(f"Target: {target}\n")
    result = []
    int_word = word_to_integer_vector(word)
    int_target = word_to_integer_vector(target)
    for i in range(len(int_word)):
        result.append(np.abs(int_word[i] - int_target[i]))
    return np.sum(result)

def crossover(subset_parents, cross_over_rate):
    print(subset_parents)
    if float(rd.randint(0, 10))/10 <= cross_over_rate:
        size_n = len(subset_parents[0])
        cut_v = random.randint(1, size_n-1)
        son1 = subset_parents[0][0:cut_v] + subset_parents[1][cut_v:]
        son2 = subset_parents[1][0:cut_v] + subset_parents[0][cut_v:]
    else:
        son1, son2 = subset_parents
    return son1, son2
 
def mutation(population, mutation_rate):
    size_ind = len(population[0])
    if float(random.randint(0, 10))*10 <= mutation_rate:
        population_copy = copy.copy(population)
        for i in len(population_copy):
            loc = random.int(0,size_ind-1)
            mut = get_repres_random()
            population_copy[i,loc] = mut
    return population_copy

words = generate_pop(2)
crossover(words, 0)

['akUEFZKEDzPcRbqoBRxUGeoYDB*D', 'yYh*R**rjdrNp***e*n***e*JGEO']


('akUEFZKEDzPcRbqoBRxUGeoYDB*D', 'yYh*R**rjdrNp***e*n***e*JGEO')

In [9]:
# gerar pop.

In [10]:
# converter de char para int (ascii)

In [6]:
# função de custo
# soma das diferenças entre os valores das duas strings

In [7]:
# mutação
# escolhe posição aleatória e troca char.

In [11]:
# selecionar os melhores indivíudos (ranking dos indivíduos)
# no caso, isso não vai ser feito
# cruzaremos todos os indivíduos
# se ficar ruim, tentamos outra abordagem: tipo ordear indivíduos e cruzar melhores entre si.

# chegamos na conclusão que não faz muito sentido, tendo em vista que não estamosa forçando seleção natural.
# vamos fazer igual fizemos nos outros 

In [62]:
# cross (reprodução)
# recebe dois indivíduos e gera dois filhos 
# um ponto de corte aleatório

In [67]:
pop = generate_pop(20)
